# Gaining insights from Unstructured Data with Snowflake Cortex AI

Tasty Bytes is a global food truck network operating in 15 countries with fleet of 450 trucks. They collect customer reviews to get customer feedback on their food-trucks which come in from multiple sources and span multiple languages. 
This enables them to better understand the areas which require improvement and drive up customer loyalty along with satisfaction. 

In this notebook, we will look at how we analyze these collated customer reviews using Snowflake Cortex to understand :
  * What our international customers are saying with Cortex **Translate**
  * Get a summary of what customers are saying with Cortex **AI_SUMMARIZE_AGG**
  * Classify reviews to determine if they would recommend a food truck with Cortex **AI_CLASSIFY**
  * Gain specific insights with Cortex **AI_COMPLETE**
  * Understand how customers are feeling with Cortex **AI_SENTIMENT**
  * Insights on Images with Cortex **AI_COMPLETE**
  * Transcribe Audio with Cortex **AI_TRANSCRIBE**

Lets see how many reviews we have.

In [ ]:
SELECT COUNT(*) FROM TRUCK_REVIEWS_V;

**Import python packages**

Snowflake Notebooks include Streamlit and the third-party packages listed in the Snowflake Anaconda channel.  
Installing a package is made easy by enabling user to select required pacakges from a list of available pacakges under Packages on the top right corner.  
Once installed, we can import installed packages as we would in any other notebook.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import json

# Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import when, date_part, ai_classify, prompt, ai_agg, ai_summarize_agg, ai_complete

# Cortex Functions
import snowflake.cortex  as cortex

session = get_active_session()
# Add a query tag to the session.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"voc", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook", "vignette":"customer_reviews"}}


In [ ]:
-- Create an image table that references the files in our stage
CREATE OR REPLACE TABLE TB_VOC.MEDIA.IMAGE_TABLE AS
  SELECT 
    RELATIVE_PATH AS image_path,
    TO_FILE('@TB_VOC.MEDIA.IMAGES', RELATIVE_PATH) AS img_file
  FROM DIRECTORY('@TB_VOC.MEDIA.IMAGES');

-- Create an audio table that references the files in our stage
CREATE OR REPLACE TABLE tb_voc.MEDIA.AUDIO_TABLE AS
  SELECT 
    RELATIVE_PATH AS audio_path,
    TO_FILE('@TB_VOC.MEDIA.AUDIO', RELATIVE_PATH) AS audio_file
  FROM DIRECTORY('@TB_VOC.MEDIA.AUDIO');

Lets preview the reviews

In [ ]:
reviews_df = session.table('TRUCK_REVIEWS_V') \
             .filter(date_part("year", F.col('DATE')) == 2024)
reviews_df.show()

In the next cell, we leverage **Translate** - one of the **Snowflake Cortex specialised LLM functions** are available in Snowpark ML, to translate the multilingual reviews to english to enable easier analysis for folks who don't speak the language the original review is in.

In [ ]:
# Conditionally translate reviews that are not english using Cortex Translate
reviews_df = reviews_df.withColumn('TRANSLATED_REVIEW',when(F.col('LANGUAGE') != F.lit("en"), \
                                                            cortex.translate(F.col('REVIEW'), \
                                                                             F.col('LANGUAGE'), \
                                                                             "en")) \
                                   .otherwise(F.col('REVIEW')))

reviews_df.filter(F.col('LANGUAGE') != F.lit("en")) \
.select(["REVIEW","LANGUAGE","TRANSLATED_REVIEW"]).show(3)

We can quickly learn what our customers are saying with Cortex **Summarize**

In [ ]:
summarized_reviews_df = session.table("TRUCK_REVIEWS_V") \
    .sample(0.01) \
    .group_by("TRUCK_BRAND_NAME") \
    .agg(ai_summarize_agg(F.col("REVIEW")).alias("SUMMARY"))

summarized_reviews_df.select(["TRUCK_BRAND_NAME", "SUMMARY"]).show(3)

We can similarly understand if a customer would recommend the food truck based on their review using Snowflake Cortex **AI_CLASSIFY**

In [ ]:
reviews_df = reviews_df.withColumn('RECOMMEND', ai_classify(prompt("Tell me based on the following food truck customer review {0}, will they recommend the food truck to their friends and family?", F.col('REVIEW')),["Likely","Unlikely","Unsure"])["labels"][0])

reviews_df.select(["REVIEW","RECOMMEND"]).show(3)

Gaining specific insights through Cortex Complete

In [ ]:
question = "What is the number one dish positively mentioned in the feedback?"

summarized_reviews_df = session.table("CONCATENATED_REVIEWS").select(
    F.col("TRUCK_BRAND_NAME"),
    ai_complete(
        "openai-gpt-4.1",
        F.concat(
            F.lit("Context: "),
            F.col("ALL_REVIEWS_TEXT"),
            F.lit(f" Question: {question} Answer briefly and concisely and only name the dish:")
        )
    ).alias("NUMBER_ONE_DISH")
)

summarized_reviews_df.show(3)

In [ ]:
image_df = session.sql("SELECT * FROM TB_VOC.MEDIA.IMAGE_TABLE").collect()

def analyze_image(image_path, prompt_template, model='claude-3-5-sonnet'):
    escaped_prompt = prompt_template.replace("'", "''")
    
    sql_query = f"""
    SELECT 
      AI_COMPLETE(
        '{model}', 
        '{escaped_prompt}',
         TO_FILE('@TB_VOC.MEDIA.IMAGES', '{image_path}')
      )
    """
    result = session.sql(sql_query).collect()
    return result[0][0]

if len(image_df) > 0:
    image_path = image_df[0]['IMAGE_PATH']  # Get first image
    prompt = "Describe what's happening in this image in the style of a detective investigating a scene. Be brief but detailed."

    analysis = analyze_image(image_path, prompt)
    print(f"Analysis for {image_path}:")
    print(analysis)
else:
    print("No images found in the table.")

In [ ]:
audio_df = session.sql("SELECT * FROM TB_VOC.MEDIA.AUDIO_TABLE").collect()

# Function to transcribe audio with different modes
def transcribe_audio(audio_path):

    sql_query = f"""
        SELECT AI_TRANSCRIBE(
            TO_FILE('@TB_VOC.MEDIA.AUDIO', '{audio_path}')
        ) as transcription_result
        """
    
    result = session.sql(sql_query).collect()
    return json.loads(result[0]['TRANSCRIPTION_RESULT'])

# Basic text transcription
if len(audio_df) > 0:
    audio_path = audio_df[0]['AUDIO_PATH']
    transcription = transcribe_audio(audio_path)
    
    print(f"Transcription for {audio_path}:")
    print(f"Duration: {transcription['audio_duration']} seconds")
    print(f"Text: {transcription['text']}")
else:
    print("No audio files found in the table.")

Next we will look at another **task specific LLM function in Cortex - Sentiment**. We utilise sentiment function to understand customer's tone based on the review they provided. Sentiment return value between -1 and 1 such that -1 is the most negative while 1 is the most positive.  

In [ ]:
# Understand the sentiment of customer review using Cortex Sentiment
reviews_df = reviews_df.withColumn('SENTIMENT', cortex.sentiment(F.col('REVIEW')))

reviews_df.select(["REVIEW","SENTIMENT"]).show(3)

These robust Cortex functions seamlessly integrate with SQL, enabling powerful data processing capabilities.

In [ ]:
-- Add the TRANSLATED_REVIEW column with conditional translation
WITH TRANSLATED_REVIEWS AS (
    SELECT 
        REVIEW,
        LANGUAGE,
        CASE 
            WHEN LANGUAGE != 'en' THEN SNOWFLAKE.CORTEX.TRANSLATE(REVIEW, LANGUAGE, 'en') 
            ELSE REVIEW
        END AS TRANSLATED_REVIEW
    FROM TRUCK_REVIEWS_V
)

-- Filter rows where the LANGUAGE is not English and select the desired columns
SELECT 
    REVIEW, 
    LANGUAGE, 
    TRANSLATED_REVIEW
FROM TRANSLATED_REVIEWS
WHERE LANGUAGE != 'en'
LIMIT 3;

In [ ]:
WITH SUMMARIZED_REVIEWS AS (
    SELECT 
        TRUCK_BRAND_NAME,
        AI_SUMMARIZE_AGG(REVIEW) AS SUMMARY
    FROM TRUCK_REVIEWS_V
    SAMPLE ROW (1)
    GROUP BY TRUCK_BRAND_NAME
)
SELECT * FROM SUMMARIZED_REVIEWS;

In [ ]:
WITH CLASSIFIED_REVIEWS AS (
    SELECT 
        REVIEW,
        AI_CLASSIFY(
            REVIEW, 
            ['Likely', 'Unlikely', 'Unsure'], 
            OBJECT_CONSTRUCT('task_description', 
                'Tell me based on the following food truck customer review, will they recommend the food truck to their friends and family?'
            )
        ):labels[0]::TEXT AS RECOMMEND
    FROM TRUCK_REVIEWS_V
)
SELECT * FROM CLASSIFIED_REVIEWS LIMIT 3;

In [ ]:
-- Gain Learnings from a specific question
WITH GAIN_LEARNINGS AS (
    SELECT 
        TRUCK_BRAND_NAME,
        AI_COMPLETE(
           'openai-gpt-4.1', 
           'Context:' || ALL_REVIEWS_TEXT || ' Question: What is the number one dish positively mentioned in the feedback? Answer briefly and concisely and only name the dish:'
       ) AS NUMBER_ONE_DISH
    FROM CONCATENATED_REVIEWS
)
SELECT TRUCK_BRAND_NAME, NUMBER_ONE_DISH FROM GAIN_LEARNINGS LIMIT 3;

In [ ]:
SELECT
  AI_COMPLETE (
    'claude-3-5-sonnet',
    'Please describe what you see in this image.',
    TO_FILE ('@TB_VOC.MEDIA.IMAGES', IMAGE_PATH)
  ) AS IMAGE_DESCRIPTION
FROM
  TB_VOC.MEDIA.IMAGE_TABLE
LIMIT
  1;

In [ ]:
SELECT
  AI_TRANSCRIBE (
    TO_FILE ('@TB_VOC.MEDIA.AUDIO', AUDIO_PATH)
  ) AS TRANSCRIPTION_RESULT
FROM
  TB_VOC.MEDIA.AUDIO_TABLE
LIMIT
  1;

In [ ]:
SELECT 
    REVIEW, 
    AI_SENTIMENT(REVIEW) AS SENTIMENT
FROM TRUCK_REVIEWS_V
LIMIT 3;
